In [13]:
%pip install ipywidgets
%pip install tensorflow
%pip install langchain-ollama
%pip install keras
%pip install numpy
%pip install pillow
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install tensorflow
%pip install kaggle


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import urllib.request
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image
from pathlib import Path
import numpy as np
import tensorflow as tf
from langchain_ollama import OllamaLLM
import ipywidgets as widgets
from IPython.display import display, clear_output
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

In [15]:

# Load the saved model
model = load_model('food_classification_model.h5')

# Load class labels
with open('class_labels.pkl', 'rb') as f:
    class_labels = pickle.load(f)

# Predict food from an image file
def predict_food(image_path):
    
    # Preprocess the image for the model
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    confidence = np.max(predictions) * 100
    
    # Return prediction and confidence
    return predicted_label, confidence

# Main execution
if __name__ == "__main__":
    input_image = str(sorted(Path("input_image").glob("*"))[0])
    label, confidence = predict_food(input_image)
    print(f"\nPredicted: {label}")
    print(f"Confidence: {confidence:.2f}%")
    print(input_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step

Predicted: apple_pie
Confidence: 23.47%
input_image\apple_pie.png


In [ ]:
# Ask the servings
servings = str(input("\nFor how many people will this be? (1-10): "))
# Ask important features
important = str(input("\nIs there anything important that you want to add before the recipe gets generated?\n\n(e.g. Allergies, Prefered ways, difficulty level, duration...)\nIf you don't have anything just press \"Enter\""))

In [ ]:
# Initialize llama3 LLM
model = OllamaLLM(model="llama3")

# Handle important features
if important.strip():
    important = 'Important feature: ' + important
else:
    important = ''

# Handle serving amount
if servings.strip() == '' or servings:
    servings = ' is for 2 people'
else:
    try:
        servings_int = int(servings)
        servings_str = f' is for {servings_int} people'
    except ValueError:
        servings_str = ' is for 2 people'

# Generate prompt
prompt=f"Make a recipe, only including the title, servings{servings}, ingredients and instructions for {label}. {important}"
result = model.invoke(input=prompt)
print(result)

Here is a recipe for Apple Pie:

**Apple Pie**

**Servings:** 2

**Ingredients:**

* 1 pie crust (homemade or store-bought)
* 6-8 medium-sized apples, peeled and sliced
* 1/2 cup granulated sugar
* 2 tablespoons all-purpose flour
* 1 teaspoon cinnamon
* 1/4 teaspoon nutmeg
* 1/4 teaspoon salt
* 1/4 cup butter, cut into small pieces

**Instructions:**

1. Preheat oven to 375°F (190°C).
2. In a large bowl, combine sliced apples, granulated sugar, flour, cinnamon, nutmeg, and salt. Mix until apples are evenly coated.
3. Roll out the pie crust to fit a 9-inch pie dish.
4. Fill the pie crust with the apple mixture and dot the top with butter pieces.
5. Roll out remaining pie crust to fit the top of the pie. Place on top of apple filling and crimp edges to seal.
6. Cut a few slits in the top crust to allow steam to escape during baking.
7. Bake for 45-50 minutes, or until crust is golden brown and apples are tender.
8. Let cool before serving. Enjoy!
